# Preparación de datos y análisis para fine-tuning de chat model

Fuente: [Data preparation and analysis for chat model fine-tuning - OpenAI Cookbook](https://cookbook.openai.com/examples/chat_finetuning_data_prep)

In [1]:
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 12.0 MB/s eta 0:00:00


In [2]:
import json
import tiktoken # para conteo de Tokens
import numpy as np
from collections import defaultdict

## Cargar dataset desde archivo JSONL

In [3]:
downloaded = False

If it is running Locally, `wget` should be installed

In [4]:
if not downloaded:
  !wget "https://raw.githubusercontent.com/pancenu/openai-api/refs/heads/main/Clase%2012-13%20Preparar%20datos%20para%20fine-tuning/data.csv"
  !wget "https://raw.githubusercontent.com/pancenu/openai-api/refs/heads/main/Clase%2012-13%20Preparar%20datos%20para%20fine-tuning/data_train.jsonl"
  !wget "https://raw.githubusercontent.com/pancenu/openai-api/refs/heads/main/Clase%2012-13%20Preparar%20datos%20para%20fine-tuning/data_val.jsonl"
  downloaded = True

--2025-04-13 15:51:47--  https://raw.githubusercontent.com/pancenu/openai-api/refs/heads/main/Clase%2012-13%20Preparar%20datos%20para%20fine-tuning/data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 667513 (652K) [text/plain]
Saving to: ‘data.csv’

data.csv            100%[===================>] 651.87K  --.-KB/s    in 0.06s   

2025-04-13 15:51:47 (11.1 MB/s) - ‘data.csv’ saved [667513/667513]

--2025-04-13 15:51:47--  https://raw.githubusercontent.com/pancenu/openai-api/refs/heads/main/Clase%2012-13%20Preparar%20datos%20para%20fine-tuning/data_train.jsonl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.1

In [5]:
data_path = "/content/data_train.jsonl"

In [6]:
# Cargar dataset
with open(data_path, 'r', encoding='utf-8') as f:
    dataset = [json.loads(line) for line in f]

# Análisis inicial del dataset
print("Num examples:", len(dataset))
print("First example:")
for message in dataset[0]["messages"]:
    print(message)

Num examples: 1261
First example:
{'role': 'system', 'content': 'Eres un asistente de atención a clientes y estudiantes de la plataforma de educación online en tecnología, inglés y liderazgo llamada Platzi'}
{'role': 'user', 'content': '¿Cómo puedo mejorar mis habilidades en JavaScript?,'}
{'role': 'assistant', 'content': 'Para mejorar tus habilidades en JavaScript, te recomendamos tomar el Curso Práctico de JavaScript en https://platzi.com/cursos/javascript-practico/. También puedes practicar en proyectos personales y buscar recursos en línea para continuar aprendiendo. ¡Mucho éxito en tu camino de aprendizaje de JavaScript!'}


## Validación de formato

In [7]:
# Revisión de errores de formato
format_errors = defaultdict(int)

for ex in dataset:
    if not isinstance(ex, dict):
        format_errors["data_type"] += 1
        continue

    messages = ex.get("messages", None)
    if not messages:
        format_errors["missing_messages_list"] += 1
        continue

    for message in messages:
        if "role" not in message or "content" not in message:
            format_errors["message_missing_key"] += 1

        if any(k not in ("role", "content", "name", "function_call") for k in message):
            format_errors["message_unrecognized_key"] += 1

        if message.get("role", None) not in ("system", "user", "assistant", "function"):
            format_errors["unrecognized_role"] += 1

        content = message.get("content", None)
        function_call = message.get("function_call", None)

        if (not content and not function_call) or not isinstance(content, str):
            format_errors["missing_content"] += 1

    if not any(message.get("role", None) == "assistant" for message in messages):
        format_errors["example_missing_assistant_message"] += 1

if format_errors:
    print("Found errors:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
else:
    print("No errors found")

No errors found


## Utilidades de conteo de Tokens

In [42]:
encoding = tiktoken.get_encoding("cl100k_base")

# not exact!
# simplified from https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb
def num_tokens_from_messages(messages, tokens_per_message=3, tokens_per_name=1):
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3
    return num_tokens

def num_assistant_tokens_from_messages(messages):
    num_tokens = 0
    for message in messages:
        if message["role"] == "assistant":
            num_tokens += len(encoding.encode(message["content"]))
    return num_tokens

def print_distribution(values, name):
    print(f"\n#### Distribution of {name}:")
    print(f"min / max: {min(values)}, {max(values)}")
    print(f"mean / median: {np.mean(values)}, {np.median(values)}")
    print(f"p5 / p95: {np.quantile(values, 0.1)}, {np.quantile(values, 0.9)}")

## Conteo de Tokens y advertencias de data

In [43]:
# Advertencias y conteo de tokens
n_missing_system = 0
n_missing_user = 0
n_messages = []
convo_lens = []
assistant_message_lens = []

for ex in dataset:
    messages = ex["messages"]
    if not any(message["role"] == "system" for message in messages):
        n_missing_system += 1
    if not any(message["role"] == "user" for message in messages):
        n_missing_user += 1
    n_messages.append(len(messages))
    convo_lens.append(num_tokens_from_messages(messages))
    assistant_message_lens.append(num_assistant_tokens_from_messages(messages))

print("Num examples missing system message:", n_missing_system)
print("Num examples missing user message:", n_missing_user)
print_distribution(n_messages, "num_messages_per_example")
print_distribution(convo_lens, "num_total_tokens_per_example")
print_distribution(assistant_message_lens, "num_assistant_tokens_per_example")
n_too_long = sum(l > 4096 for l in convo_lens)
print(f"\n{n_too_long} examples may be over the 4096 token limit, they will be truncated during fine-tuning")

Num examples missing system message: 0
Num examples missing user message: 0

#### Distribution of num_messages_per_example:
min / max: 3, 3
mean / median: 3.0, 3.0
p5 / p95: 3.0, 3.0

#### Distribution of num_total_tokens_per_example:
min / max: 73, 190
mean / median: 134.2140221402214, 134.0
p5 / p95: 114.0, 157.0

#### Distribution of num_assistant_tokens_per_example:
min / max: 10, 118
mean / median: 69.91512915129151, 70.0
p5 / p95: 50.0, 92.0

0 examples may be over the 4096 token limit, they will be truncated during fine-tuning


## Estimación de costos

In [44]:
# Costo y estimado de número de épocas por defecto
MAX_TOKENS_PER_EXAMPLE = 4096

TARGET_EPOCHS = 3
MIN_TARGET_EXAMPLES = 100
MAX_TARGET_EXAMPLES = 25000
MIN_DEFAULT_EPOCHS = 1
MAX_DEFAULT_EPOCHS = 25

n_epochs = TARGET_EPOCHS
n_train_examples = len(dataset)
if n_train_examples * TARGET_EPOCHS < MIN_TARGET_EXAMPLES:
    n_epochs = min(MAX_DEFAULT_EPOCHS, MIN_TARGET_EXAMPLES // n_train_examples)
elif n_train_examples * TARGET_EPOCHS > MAX_TARGET_EXAMPLES:
    n_epochs = max(MIN_DEFAULT_EPOCHS, MAX_TARGET_EXAMPLES // n_train_examples)

n_billing_tokens_in_dataset = sum(min(MAX_TOKENS_PER_EXAMPLE, length) for length in convo_lens)
print(f"Dataset has ~{n_billing_tokens_in_dataset} tokens that will be charged for during training")
print(f"By default, you'll train for {n_epochs} epochs on this dataset")
print(f"By default, you'll be charged for ~{n_epochs * n_billing_tokens_in_dataset} tokens")

Dataset has ~36372 tokens that will be charged for during training
By default, you'll train for 3 epochs on this dataset
By default, you'll be charged for ~109116 tokens
